# PANDA HANDLER

In [47]:
import pandas as pd

In [5]:
df = pd.read_excel('BMF2018.xltx')

# PEGANDO BLUE CHIPS

In [90]:
bmf2018 = df
#bmf2018

In [93]:
# PRIMEITO AGRUPA DE ACORDO COM ATIVO
df_gb = bmf2018.groupby('CODIGO DE NEGOGIACAO DO PAPEL')

# DEPOIS SOMA O VOLUME E ORDENAR DECRESCENTE
bc = df_gb.sum()['VOLUME TOTAL DE TITULOS NEGOCIADOS NESTE PAPEL'].sort_values(ascending=False)

# CRIA INDICE PARA O NOME DO ATIVO DEIXAR DE SER O INDICE
blue_chips = bc.reset_index()['CODIGO DE NEGOGIACAO DO PAPEL']

# SELECIONA SOMENTE OS 100 ATIVOS QUE TEM NOME PADRAO (EXCLUI MERCADO A TERMO, FIIs, ETC)  
blue_chips = blue_chips[blue_chips.apply(lambda name: len(name) <= 5 )].iloc[:100]
blue_chips.nunique()
# SALVA NUM CSV
blue_chips.to_csv('BLUECHIPS',index=False)

/home/rodrigo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  


In [94]:
blue_chips.nunique()


100

In [21]:
# PERAI



In [18]:
df = pd.read_csv('banklist.csv')
banks = df

** Show the head of the dataframe **

In [19]:
banks.tail()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
546,"Superior Bank, FSB",Hinsdale,IL,32646,"Superior Federal, FSB",27-Jul-01,19-Aug-14
547,Malta National Bank,Malta,OH,6629,North Valley Bank,3-May-01,18-Nov-02
548,First Alliance Bank & Trust Co.,Manchester,NH,34264,Southern New Hampshire Bank & Trust,2-Feb-01,18-Feb-03
549,National State Bank of Metropolis,Metropolis,IL,3815,Banterra Bank of Marion,14-Dec-00,17-Mar-05
550,Bank of Honolulu,Honolulu,HI,21029,Bank of the Orient,13-Oct-00,17-Mar-05


** What are the column names? **

In [8]:
banks.columns

Index(['Bank Name', 'City', 'ST', 'CERT', 'Acquiring Institution',
       'Closing Date', 'Updated Date'],
      dtype='object')

Index(['Bank Name', 'City', 'ST', 'CERT', 'Acquiring Institution',
       'Closing Date', 'Updated Date'],
      dtype='object')

** How many States (ST) are represented in this data set? **

In [11]:
banks['ST'].unique().size

44

44

** Get a list or array of all the states in the data set. **

In [12]:
banks['ST'].unique()

array(['IL', 'WI', 'LA', 'UT', 'NJ', 'AR', 'GA', 'PA', 'TN', 'WA', 'CO',
       'PR', 'FL', 'MN', 'CA', 'MD', 'OK', 'OH', 'SC', 'VA', 'ID', 'TX',
       'CT', 'AZ', 'NV', 'NC', 'KY', 'MO', 'KS', 'AL', 'MI', 'IN', 'IA',
       'NE', 'MS', 'NM', 'OR', 'NY', 'MA', 'SD', 'WY', 'WV', 'NH', 'HI'],
      dtype=object)

array(['IL', 'WI', 'LA', 'UT', 'NJ', 'AR', 'GA', 'PA', 'TN', 'WA', 'CO',
       'PR', 'FL', 'MN', 'CA', 'MD', 'OK', 'OH', 'SC', 'VA', 'ID', 'TX',
       'CT', 'AZ', 'NV', 'NC', 'KY', 'MO', 'KS', 'AL', 'MI', 'IN', 'IA',
       'NE', 'MS', 'NM', 'OR', 'NY', 'MA', 'SD', 'WY', 'WV', 'NH', 'HI'], dtype=object)

** What are the top 5 states with the most failed banks? **

In [22]:
banks.groupby('ST').count().sort_values('Bank Name',ascending=False).iloc[:5]['Bank Name']

ST
GA    93
FL    75
IL    67
CA    41
MN    23
Name: Bank Name, dtype: int64

ST
GA    93
FL    75
IL    67
CA    41
MN    23
Name: Bank Name, dtype: int64

** What are the top 5 acquiring institutions? **

In [28]:
banks['Acquiring Institution'].value_counts().iloc[1:6]

State Bank and Trust Company           12
First-Citizens Bank & Trust Company    11
Ameris Bank                            10
U.S. Bank N.A.                          9
Community & Southern Bank               8
Name: Acquiring Institution, dtype: int64

No Acquirer                            31
State Bank and Trust Company           12
First-Citizens Bank & Trust Company    11
Ameris Bank                            10
U.S. Bank N.A.                          9
Name: Acquiring Institution, dtype: int64

** How many banks has the State Bank of Texas acquired? How many of them were actually in Texas?**

In [30]:
banks[banks['Acquiring Institution'] == 'State Bank of Texas']

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
4,Seaway Bank and Trust Company,Chicago,IL,19328,State Bank of Texas,27-Jan-17,18-May-17
21,The National Republic Bank of Chicago,Chicago,IL,916,State Bank of Texas,24-Oct-14,6-Jan-16
450,Millennium State Bank of Texas,Dallas,TX,57667,State Bank of Texas,2-Jul-09,26-Oct-12


,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
4,Seaway Bank and Trust Company,Chicago,IL,19328,State Bank of Texas,27-Jan-17,18-May-17
21,The National Republic Bank of Chicago,Chicago,IL,916,State Bank of Texas,24-Oct-14,6-Jan-16
450,Millennium State Bank of Texas,Dallas,TX,57667,State Bank of Texas,2-Jul-09,26-Oct-12


** What is the most common city in California for a bank to fail in?**

In [41]:
banks[banks['ST']=='CA'].groupby('City').count().sort_values('Bank Name',ascending=False).head(1)

,Bank Name,ST,CERT,Acquiring Institution,Closing Date,Updated Date
City,,,,,,
Los Angeles,4,4,4,4,4,4


,Bank Name,ST,CERT,Acquiring Institution,Closing Date,Updated Date
City,,,,,,
Los Angeles,4,4,4,4,4,4


** How many failed banks don't have the word "Bank" in their name? **

In [43]:
banks['Bank Name'].apply(lambda name: 'Bank' not in name).sum()

14

14

** How many bank names start with the letter 's' ? **

In [45]:
banks['Bank Name'].apply(lambda name: 'S' == name[0].upper()).sum()

53

53

** How many CERT values are above 20000 ? **

In [47]:
(banks['CERT'] > 20000).sum()

417

417

** How many bank names consist of just two words? (e.g. "First Bank" , "Bank Georgia" )**

In [48]:
banks['Bank Name'].apply(lambda name: len(name.split())==2 ).sum()

114

114

**Bonus: How many banks closed in the year 2008? (this is hard because we technically haven't learned about time series with pandas yet! Feel free to skip this one!**

In [51]:
banks['Closing Date'].apply(lambda date: date[-2:] == '08' ).sum()

25

25

# GREAT JOB!